# Queremos crear un bucket S3 en nuesto laboratorio de AWS Academy, con datos de un faker

## Primero — Introducir credenciales de AWS Academy y crear sesión boto3

In [14]:
# === CELDA 1: Credenciales AWS Academy (temporales) + sesión boto3 ===
!pip -q install boto3 faker

import os
import getpass
import boto3

# Pide credenciales por consola (en Colab no se verán secret/token si usas getpass)
AWS_ACCESS_KEY_ID = input("AWS Access Key ID: ").strip()
AWS_SECRET_ACCESS_KEY = getpass.getpass("AWS Secret Access Key (no se mostrará): ").strip()
AWS_SESSION_TOKEN = getpass.getpass("AWS Session Token (no se mostrará): ").strip()

# Región (en AWS Academy suele variar; pon la que te indique el Lab o usa la que necesites)
AWS_REGION = input("AWS Region (ej: us-east-1): ").strip() or "us-east-1"

# Guardarlas como variables de entorno (boto3 las detecta automáticamente)
os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
os.environ["AWS_SESSION_TOKEN"] = AWS_SESSION_TOKEN
os.environ["AWS_DEFAULT_REGION"] = AWS_REGION

# Crea sesión/cliente para verificar que autentica
session = boto3.Session(region_name=AWS_REGION)
sts = session.client("sts")
identity = sts.get_caller_identity()

print("Credenciales cargadas correctamente.")
print("Cuenta:", identity.get("Account"))
print("ARN:", identity.get("Arn"))
print("Región:", AWS_REGION)


Credenciales cargadas correctamente.
Cuenta: 199824145583
ARN: arn:aws:sts::199824145583:assumed-role/voclabs/user4579198=Diego
Región: us-east-1


## Segundo — Generar Faker y subir JSON al bucket

In [16]:
# === CELDA 2: Generar Faker + subir a S3 ===
import json
import random
import time
from datetime import datetime, timezone
from faker import Faker
from botocore.exceptions import ClientError

fake = Faker()

BUCKET_NAME = "iabd03-tarea-ra2"
PREFIX = "data/sensores/"
FILE_NAME_LOCAL = "iabd03_sensores.json"

# Creamos una pequeña "flota" de 5 sensores fijos y legibles
SENSORES_DISPONIBLES = ["SENS-01", "SENS-02", "SENS-03", "SENS-04", "SENS-05"]

def generar_datos():
    return {
        "timestamp": fake.date_time_between(start_date='-7d', end_date='now', tzinfo=timezone.utc).isoformat(),
        "sensor_id": random.choice(SENSORES_DISPONIBLES),
        "sensor_state": random.choice(["OK", "WARN", "FAIL"]),
        "temperature_c": round(random.uniform(15.0, 40.0), 2),
        "co2_ppm": round(random.uniform(300, 800), 2),
        "lat": round(random.uniform(-90, 90), 6),
        "lon": round(random.uniform(-180, 180), 6)
    }

N = int(input("¿Cuántos registros generar? (ej: 50): ").strip() or "50")
datos = [generar_datos() for _ in range(N)]

# Guardar localmente
with open(FILE_NAME_LOCAL, "w", encoding="utf-8") as f:
    json.dump(datos, f, ensure_ascii=False)

# Clave destino en S3
S3_KEY = f"{PREFIX}{FILE_NAME_LOCAL}" if "PREFIX" in globals() else FILE_NAME_LOCAL

# Subir a S3
s3 = session.client("s3", region_name=AWS_REGION)
try:
    s3.upload_file(FILE_NAME_LOCAL, BUCKET_NAME, S3_KEY)
    print(f"Subido a s3://{BUCKET_NAME}/{S3_KEY}")
    
    # Borrar el archivo local tras la subida exitosa
    if os.path.exists(FILE_NAME_LOCAL):
        os.remove(FILE_NAME_LOCAL)
        print(f"Archivo local '{FILE_NAME_LOCAL}' eliminado correctamente")
    
except ClientError as e:
    code = e.response.get("Error", {}).get("Code", "Unknown")
    msg = e.response.get("Error", {}).get("Message", str(e))
    raise RuntimeError(f"Error subiendo a S3 ({code}): {msg}") from e


Subido a s3://iabd03-tarea-ra2/data/sensores/iabd03_sensores.json
Archivo local 'iabd03_sensores.json' eliminado correctamente
